In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import RFECV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classificatin_report
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/데이터스쿨/실습파일/ML_project/dataset.csv', index_col=0)
exited = {'Existing Customer': 0, 'Attrited Customer':1}
gender = {'M':0, 'F':1}
education = {'Uneducated': 0, 'High School':1, 'College':2, 'Graduate':3, 'Post-Graduate': 4, 'Doctorate':5, 'Unknown':6}
marital = {'Single':0, 'Married': 1, 'Divorced':2, 'Unknown':3}
income = {'Less than $40K':0, '$40K - $60K':1, '$60K - $80K':2, '$80K - $120K':3, '$120K +':4, 'Unknown':5}
card = {'Blue':0, 'Silver':1 , 'Gold': 2, 'Platinum': 3}
df['Exited'] = df['Exited'].map(exited)
df['Gender'] = df['Gender'].map(gender)
df['Education'] = df['Education'].map(education)
df['Marital'] = df['Marital'].map(marital)
df['Income'] = df['Income'].map(income)
df['Card_type'] = df['Card_type'].map(card)
df.head()

,Exited,Age,Gender,Dependents,Education,Marital,Income,Card_type,Tenure,Product_Cnt,Inactive_Months,Contacts_Cnt,Credit_Limit,Revolv_Bal,Avg_OTB,Avg_Util_Ratio,Trans_Amt,Trans_Cnt,Amt_Chng_Q4_Q1,Cnt_Chng_Q4_Q1
0,0,45,0,3,1,1,2,0,39,5,1,3,12691.0,777,11914.0,0.061,1144,42,1.335,1.625
1,0,49,1,5,3,0,0,0,44,6,1,2,8256.0,864,7392.0,0.105,1291,33,1.541,3.714
2,0,51,0,3,3,1,3,0,36,4,1,0,3418.0,0,3418.0,0.000,1887,20,2.594,2.333
3,0,40,1,4,1,3,0,0,34,3,4,1,3313.0,2517,796.0,0.760,1171,20,1.405,2.333
4,0,40,0,3,0,1,2,0,21,5,1,0,4716.0,0,4716.0,0.000,816,28,2.175,2.500


In [ ]:
X = df[['Product_Cnt', 'Inactive_Months', 'Contacts_Cnt', 'Revolv_Bal', 'Avg_Util_Ratio', 'Trans_Amt', 'Trans_Cnt', 'Amt_Chng_Q4_Q1','Cnt_Chng_Q4_Q1']]
y = df['Exited']
rf_clf = RandomForestClassifier(random_state=13, n_jobs=-1)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)
estimators = [('scaler', StandardScaler()), ('clf', rf_clf)]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13, stratify=y)

pipe = Pipeline(estimators)

params = [{'clf__n_estimators': [100, 200, 300, 400],
           'clf__max_depth': [3, 5, 7, 9],
           'clf__min_samples_split': [2, 5, 10],
           'clf__min_samples_leaf' : [1, 2, 4]
           }]


GridSearch = GridSearchCV(estimator=pipe, param_grid=params, cv=skf)
GridSearch.fit(X_train, y_train)

score_df = pd.DataFrame(GridSearch.cv_results_)
score_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__max_depth,param_clf__min_samples_leaf,param_clf__min_samples_split,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.721277,0.146924,0.051734,0.002675,3,1,2,100,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.896360,0.903086,0.904938,0.903086,0.893827,0.900260,0.004346,109
1,1.207719,0.467896,0.071508,0.009869,3,1,2,200,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.895743,0.903704,0.902469,0.901852,0.894444,0.899643,0.003784,127
2,1.625714,0.469115,0.120287,0.018016,3,1,2,300,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.894510,0.901235,0.899383,0.901852,0.894444,0.898285,0.003213,136
3,2.009175,0.429425,0.150741,0.011438,3,1,2,400,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.895126,0.902469,0.900617,0.902469,0.897531,0.899643,0.002891,118
4,0.746923,0.110826,0.047004,0.004724,3,1,5,100,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.896360,0.903086,0.904938,0.903086,0.893827,0.900260,0.004346,109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,3.336275,0.813353,0.156518,0.005936,9,4,5,400,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.950648,0.957407,0.959877,0.960494,0.954938,0.956673,0.003598,23
140,0.725988,0.011896,0.046114,0.004277,9,4,10,100,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.947563,0.951852,0.959259,0.959877,0.953704,0.954451,0.004633,36
141,1.851647,0.487465,0.090127,0.008753,9,4,10,200,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.948797,0.955556,0.959259,0.959259,0.956173,0.955809,0.003826,35
142,2.587766,0.772147,0.128606,0.016390,9,4,10,300,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.950031,0.958025,0.959259,0.959877,0.956173,0.956673,0.003554,22


In [ ]:
score_df.sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_clf__max_depth,param_clf__min_samples_leaf,param_clf__min_samples_split,param_clf__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
112,1.049603,0.315755,0.048520,0.000732,9,1,5,100,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.951882,0.962963,0.959259,0.963580,0.956173,0.958771,0.004362,1
122,2.614749,0.470628,0.123251,0.009649,9,2,2,300,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.953115,0.958642,0.961111,0.962346,0.956790,0.958401,0.003271,2
120,0.771619,0.020949,0.046596,0.004551,9,2,2,100,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.953115,0.959259,0.961728,0.962963,0.954938,0.958401,0.003809,3
111,3.453593,0.658435,0.176723,0.029739,9,1,2,400,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.953732,0.959259,0.958642,0.963580,0.956790,0.958401,0.003225,4
121,1.889027,0.545407,0.090534,0.011316,9,2,2,200,"{'clf__max_depth': 9, 'clf__min_samples_leaf':...",0.951882,0.959877,0.959259,0.963580,0.956173,0.958154,0.003922,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22,1.281538,0.042470,0.105482,0.009671,3,2,10,300,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.894510,0.901235,0.899383,0.901852,0.894444,0.898285,0.003213,136
30,1.811022,0.599881,0.127579,0.025297,3,4,5,300,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.894510,0.901235,0.899383,0.901852,0.894444,0.898285,0.003213,136
2,1.625714,0.469115,0.120287,0.018016,3,1,2,300,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.894510,0.901235,0.899383,0.901852,0.894444,0.898285,0.003213,136
14,1.537382,0.401544,0.118595,0.004607,3,2,2,300,"{'clf__max_depth': 3, 'clf__min_samples_leaf':...",0.894510,0.901235,0.899383,0.901852,0.894444,0.898285,0.003213,136


### best parameter & score

In [ ]:
print(GridSearch.best_params_)
print(GridSearch.best_score_)

{'clf__max_depth': 9, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 5, 'clf__n_estimators': 100}
0.9587713726475806


### 베스트 랜덤포레스트 평가지표

In [ ]:
best = GridSearch.best_estimator_
pred = best.predict(X_test)
print('accuracy: ', accuracy_score(y_test, pred))
print('precision: ', precision_score(y_test, pred))
print('recall: ', recall_score(y_test, pred))
print('f1: ', f1_score(y_test, pred))

accuracy:  0.9550839091806516
precision:  0.8823529411764706
recall:  0.8307692307692308
f1:  0.8557844690966719
